# Synthetic Data Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/synthetic_data_generation/synthetic_data_generation.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/synthetic_data_generation/synthetic_data_generation.ipynb)

## Business Scenario

Your new Orders pipeline is defined and ready to go — but the upstream source system won't have
real data available for another two weeks. You need to:

1. **Unblock your team** so pipeline development can proceed in parallel.
2. **Stress-test your quarantine logic** with records that deliberately break your quality rules.
3. **Benchmark engine performance** at realistic volumes (e.g., 500k rows) before going live.

The `DataGenerator` does all three — it reads your contract and produces data that respects your
field types, `accepted_values`, and `range` rules, with no manual CSV authoring required.

## Business Value

| Without DataGenerator | With DataGenerator |
|---|---|
| Wait for real data to arrive | Start immediately — contract is the spec |
| Hand-craft CSV test files | Generated automatically from your YAML |
| Hope quarantine catches bad rows | Inject 10% invalid rows and *verify* it |
| No way to benchmark locally | Generate 500k rows in seconds |

---

## Goals

1. Generate valid synthetic orders from the contract
2. Inspect what the generator creates for each field type
3. Inject intentionally invalid rows and run them through the pipeline
4. Confirm quarantine captures exactly the bad rows
5. Scale test: generate 100k rows and measure time


In [ ]:
from pathlib import Path
from lakelogic import DataGenerator, DataProcessor, infer_contract

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

EXAMPLE_DIR = resolve_example_dir("synthetic_data_generation")
(EXAMPLE_DIR / "data").mkdir(exist_ok=True)

def contract_path(name: str) -> Path:
    return EXAMPLE_DIR / name

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "head"):
        try:
            print(df.head(limit))
            return
        except Exception:
            pass
    print(df)

print(f"Working directory: {EXAMPLE_DIR}")

## 1. Generate Valid Synthetic Orders

The simplest use-case: generate 200 rows that fully satisfy the contract rules.
No real data needed — the contract YAML is your spec.

Notice how the generator:
- Picks `status` and `region` from their `accepted_values` lists
- Keeps `amount` within the `range` constraint (0.01 → 50,000)
- Uses realistic emails and names via Faker (if installed)
- Generates `order_date` as a proper ISO date string

In [ ]:
# Instantiate the generator — seed=42 makes output reproducible
gen = DataGenerator(contract_path("contract_orders.yaml"), seed=42)

# Generate 200 fully-valid rows
df_valid = gen.generate(rows=200)

print(f"Shape: {df_valid.shape}")
print(f"Columns: {df_valid.columns}")
print()
preview_frame(df_valid, title="Sample valid orders:", limit=8)

## 2. Inspect Field-Level Distribution

Check that the generator honours the contract constraints — status should only show
accepted values, amount should stay in range, etc.

In [ ]:
import polars as pl

print("=== status distribution ===")
print(df_valid["status"].value_counts().sort("count", descending=True))

print("\n=== region distribution ===")
print(df_valid["region"].value_counts().sort("count", descending=True))

print("\n=== amount stats ===")
print(df_valid.select([
    pl.col("amount").min().alias("min"),
    pl.col("amount").max().alias("max"),
    pl.col("amount").mean().round(2).alias("mean"),
]))

print("\n=== null counts ===")
print(df_valid.null_count())

## 3. Save to Disk

Generated data can be saved as Parquet, CSV, or JSON — ready to be used
as the source input for a `DataProcessor` run.

In [ ]:
# Save as Parquet (default, recommended)
parquet_path = EXAMPLE_DIR / "data" / "orders_sample.parquet"
gen.save(df_valid, parquet_path, format="parquet")

# Also save a CSV for inspection
csv_path = EXAMPLE_DIR / "data" / "orders_sample.csv"
gen.save(df_valid, csv_path, format="csv")

print(f"Parquet saved: {parquet_path} ({parquet_path.stat().st_size:,} bytes)")
print(f"CSV saved:     {csv_path} ({csv_path.stat().st_size:,} bytes)")

## 4. Stress-Test Quarantine with Invalid Rows

This is the most powerful use-case.

We generate **500 rows with 15% intentionally invalid** — amounts below 0,
status values not in the accepted list, etc. Then we run them through the pipeline.

**Expected outcome:** The quarantine layer catches the bad rows, the good rows
pass through, and every rejected record includes an explanation of why it failed.
This verifies your quality rules are working *before* real data arrives.

In [ ]:
# Generate 500 rows: 85% valid, 15% intentionally rule-breaking
df_mixed = gen.generate(rows=500, invalid_ratio=0.15)

mixed_path = EXAMPLE_DIR / "data" / "orders_mixed.parquet"
gen.save(df_mixed, mixed_path, format="parquet")

print(f"Total rows generated: {len(df_mixed)}")
print(f"Expected valid: ~{int(500 * 0.85)}  |  Expected invalid: ~{int(500 * 0.15)}")
print(f"Saved to: {mixed_path}")

In [ ]:
# Now run the mixed dataset through the DataProcessor
processor = DataProcessor(contract=contract_path("contract_orders.yaml"))
result = processor.run_source(str(mixed_path))

good_df  = result.good
bad_df   = result.bad

total    = len(df_mixed)
n_good   = len(good_df) if good_df is not None else 0
n_bad    = len(bad_df)  if bad_df  is not None else 0

print(f"\nResults")
print(f"  Total input rows : {total}")
print(f"  Passed (good)    : {n_good}  ({n_good/total:.1%})")
print(f"  Quarantined (bad): {n_bad}   ({n_bad/total:.1%})")

In [ ]:
# Inspect why rows were quarantined
# The _lakelogic_error_reason column shows exactly which rule failed
if bad_df is not None and len(bad_df) > 0:
    reason_col = "_lakelogic_error_reason"
    if reason_col in bad_df.columns:
        print("Quarantine failure reasons (sample):")
        print(bad_df.select([c for c in bad_df.columns if c != "__index_level_0__"]).head(10))
    else:
        print("Quarantined rows (sample):")
        print(bad_df.head(10))
else:
    print("No quarantined rows — all rows passed!")

## 5. Pandas Output

If you prefer working in Pandas (e.g., for EDA or scikit-learn integration),
just switch the `output_format`:

In [ ]:
df_pandas = gen.generate(rows=50, output_format="pandas")

print(f"Type: {type(df_pandas)}")
print(f"Shape: {df_pandas.shape}")
print()
print(df_pandas.head(5).to_string(index=False))

## 6. Scale Test — 100k Rows

Generate a large dataset to benchmark your pipeline before going to production.
This also tests whether your contract can handle the expected data volumes.

In [ ]:
import time

t0 = time.perf_counter()
df_large = gen.generate(rows=100_000)
t1 = time.perf_counter()

large_path = EXAMPLE_DIR / "data" / "orders_100k.parquet"
gen.save(df_large, large_path, format="parquet")
t2 = time.perf_counter()

print(f"Generated 100,000 rows in {t1 - t0:.2f}s")
print(f"Saved to parquet in     {t2 - t1:.2f}s")
print(f"File size: {large_path.stat().st_size / 1024 / 1024:.2f} MB")
print(f"Schema:")
print(df_large.schema)

## 7. CLI Equivalent

Everything above is also available from the terminal — useful for CI pipelines
or scripted test harnesses:

```bash
# Generate 1,000 valid rows
lakelogic generate --contract contract_orders.yaml --rows 1000 --output data/orders_sample.parquet

# 500 rows with 15% invalid to stress-test quarantine
lakelogic generate --contract contract_orders.yaml --rows 500 \
    --invalid-ratio 0.15 --format csv --output data/orders_mixed.csv

# Reproducible run (same seed = same data)
lakelogic generate --contract contract_orders.yaml --rows 200 \
    --seed 42 --output data/orders_fixed.parquet --preview 10
```


---

## 8. Seed Generation from a Real File — `generate_from_sample()`

You already have a contract **and** a sample of real data.
`generate_from_sample()` pulls its value pools directly from the file, so
generated rows reflect the **same distributions** as your real data — same
statuses, same regions, same customer names — rather than random strings.

The contract still drives **types**, **required** flags, and **quality rules**;
the file drives **what values actually appear**.


In [ ]:
# Contract-backed generator + real CSV as value pool
gen_seeded = DataGenerator(contract_path("contract_orders.yaml"), seed=42)

csv_seed     = EXAMPLE_DIR / "data" / "orders_sample.csv"
parquet_seed = EXAMPLE_DIR / "data" / "orders_sample.parquet"

df_seeded = gen_seeded.generate_from_sample(csv_seed, rows=300)

print(f"Shape: {df_seeded.shape}")
print("\nStatus values (from the real file):")
print(sorted(df_seeded["status"].drop_nulls().unique().to_list()))
print("\nRegion values:")
print(sorted(df_seeded["region"].drop_nulls().unique().to_list()))

preview_frame(df_seeded, title="\nSample (file-seeded):", limit=5)

In [ ]:
# Seed only specific columns — others fall back to contract-driven generation
df_partial = gen_seeded.generate_from_sample(
    parquet_seed,
    rows=200,
    columns=["status", "region"],   # only these use real-file values
    invalid_ratio=0.1,              # still supports bad-row injection
)

print(f"Shape: {df_partial.shape}")
print("Unique status:", sorted(df_partial["status"].drop_nulls().unique().to_list()))
preview_frame(df_partial, title="\nPartial seed (status + region only):", limit=5)

---

## 9. No Contract? No Problem — `DataGenerator.from_file()`

You have a data file but **no contract at all**. `DataGenerator.from_file()`
infers the schema automatically and starts generating immediately.

Ideal for:
- Quickly augmenting an existing dataset with more rows
- Prototyping before writing a formal contract
- Ad-hoc data generation in a notebook


In [ ]:
# No contract file — schema inferred entirely from the CSV
gen_auto = DataGenerator.from_file(csv_seed, seed=7)

df_auto = gen_auto.generate(rows=500)

print(f"Shape: {df_auto.shape}")
print(f"Columns: {df_auto.columns}")
print("\nStatus values:", sorted(df_auto["status"].drop_nulls().unique().to_list()))

preview_frame(df_auto, title="\nAuto-inferred, no contract:", limit=5)

In [ ]:
# from_file also accepts an existing in-memory Polars or Pandas DataFrame
import polars as pl

df_mem     = pl.read_parquet(parquet_seed)
gen_mem    = DataGenerator.from_file(df_mem, seed=0)

df_from_df = gen_mem.generate(rows=100, invalid_ratio=0.2)
print(f"Shape: {df_from_df.shape}  (includes 20% deliberately invalid rows)")
preview_frame(df_from_df, limit=5)

---

## 10. Infer & Save a Contract — `infer_contract()`

When you want to **persist** the inferred schema as a proper contract YAML,
`infer_contract()` returns a `ContractDraft` you can inspect, edit, and save.

Once saved, it becomes a first-class LakeLogic contract usable with
`DataProcessor`, `DataGenerator`, the CLI, and the registry driver.


In [ ]:
# Infer a contract from the sample CSV
draft = infer_contract(
    csv_seed,
    title="Orders (Inferred)",
    layer="bronze",
    suggest_rules=True,    # auto-suggest accepted_values, not_null, range rules
)

print(draft)
print(f"\nInferred {len(draft.fields)} fields")
print(f"Quality rules: {len(draft.quality.get('row_rules', []))} row rules")
print()
draft.show()    # print the full YAML contract

In [ ]:
# Save the inferred contract to disk
inferred_contract_path = EXAMPLE_DIR / "data" / "contract_orders_inferred.yaml"
draft.save(inferred_contract_path)
print(f"Saved: {inferred_contract_path}")

# Chain directly into a generator — no intermediate save needed
gen_inferred = draft.to_generator(seed=99)
df_inferred  = gen_inferred.generate(rows=200)

print(f"Generated {len(df_inferred)} rows from inferred contract")
print(f"Columns: {df_inferred.columns}")

In [ ]:
# Shortest path: file → inferred contract → generated data (all in-memory)
df_chain = (
    infer_contract(csv_seed)
    .to_generator(seed=42)
    .generate(rows=1_000)
)

print(f"Chain result: {df_chain.shape}")
preview_frame(df_chain, title="First 5 rows:", limit=5)

---

## Summary

| Scenario | API | Contract needed? |
|---|---|---|
| Generate from a YAML contract | `DataGenerator(contract).generate(rows=N)` | ✅ Yes |
| Inject bad rows for quarantine testing | `gen.generate(rows=N, invalid_ratio=0.15)` | ✅ Yes |
| Seed value pools from a real file | `gen.generate_from_sample(file, rows=N)` | ✅ Yes |
| Seed only specific columns | `gen.generate_from_sample(file, rows=N, columns=[...])` | ✅ Yes |
| No contract — infer schema from file | `DataGenerator.from_file(file).generate(rows=N)` | ❌ No |
| No contract — from a DataFrame | `DataGenerator.from_file(df).generate(rows=N)` | ❌ No |
| Inspect / save inferred contract | `infer_contract(file).show()` / `.save(path)` | ❌ No |
| One-liner: file → contract → data | `infer_contract(file).to_generator().generate(rows=N)` | ❌ No |
| Pandas output | `gen.generate(output_format="pandas")` | Either |
| Reproducible output | `DataGenerator(contract, seed=42)` | Either |

### When to use each

- **Pure contract** — building a new pipeline and no data exists yet.
- **Contract + file seed** — real data exists; generated rows should look realistic, not random.
- **`from_file()` / `infer_contract()`** — data exists but no contract; start fast, formalise later.
